In [ ]:
import pandas as pd
import numpy as np
from scipy import interpolate
import plotly.graph_objects as go

## Loading Data

In [ ]:
filename = '../data/2020-03-10_QPD-Tilt-Calibration_Test1.csv'
col_names = ['theta_x', 'theta_y', 'qpd_x', 'qpd_y', 'qpd_sum']
df = pd.read_csv(filename, names=col_names)

### Check Data Validity

In [ ]:
df.head()

In [ ]:
theta_x = df['theta_x']
theta_y = df['theta_y']
theta_z = df['qpd_x']

points = np.array([df['theta_x'], df['theta_y']]).transpose()
#print(points.shape)

values = df['qpd_x']
#print(values.shape)

grid_x, grid_y = np.mgrid[-0.5:0.5:100j, -0.4:0.9:100j]
#print(grid_x.shape)

znew = interpolate.griddata(points, values, (grid_x, grid_y), method='linear')
#print(znew.shape)

fig = go.Figure()

fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers'))


fig.add_trace(go.Surface(z=znew, x=grid_x, y=grid_y))
fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

## Surface Fitting
Fit 2D Functions of form $q_x=f(\theta_x, \theta_y)$ and $q_y=f(\theta_x, \theta_y)$

e.g. using 3rd order polynomials:

$q_x = a_1.\theta_x^3 + a_2.\theta_y^3 + a_3.\theta_x^2 + a_4.\theta_y^2 + a_5.\theta_x + a_6.\theta_y + a_7$

$q_y = b_1.\theta_x^3 + b_2.\theta_y^3 + b_3.\theta_x^2 + b_4.\theta_y^2 + b_5.\theta_x + b_6.\theta_y + b_7$

In [ ]:
def surface_fitting(x, y, z):
    X = x.to_numpy().flatten()
    Y = y.to_numpy().flatten()
    Z = z.to_numpy().flatten()

    A = np.array([X**3, Y**3, X**2, Y**2, X, Y, X*0+1]).T
    #print(A.shape)

    coeff, r, rank, s = np.linalg.lstsq(A, Z, rcond=None)

    return coeff

In [ ]:
c = surface_fitting(df['theta_x'], df['theta_y'], df['qpd_x'])
print(c)

In [ ]:
def poly2Dreco(X, Y, c):
    return (c[0]*X**3 + c[1]*Y**3 + c[2]*X**2 + c[3]*Y**2 + c[4]*X + c[5]*Y + c[6])

def plot_fitted_surface(x, y, z, c):
    grid_x, grid_y = np.mgrid[-0.5:0.5:100j, -0.4:0.9:100j]

    zfit = poly2Dreco(grid_x, grid_y, c)

    fig = go.Figure()

    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers'))


    fig.add_trace(go.Surface(z=zfit, x=grid_x, y=grid_y))
                    
    fig.update_layout(scene = dict(
                        xaxis_title='X AXIS TITLE',
                        yaxis_title='Y AXIS TITLE',
                        zaxis_title='Z AXIS TITLE'))
    fig.show()

In [ ]:
plot_fitted_surface(df['theta_x'], df['theta_y'], df['qpd_x'], c)


## Calibration Function

Create function that calculates $q_x$ an $q_y$ from $\theta_x$ and $\theta_y$

In [ ]:
def my_function(z, c):
    theta_x = z[0]
    theta_y = z[1]

    F = np.zeros_like(z)
    for i in range(0, 1):
        F[0] = c[i, 0]*theta_x**3 + c[i, ]*theta_y**3 + c[i, 2]*theta_x**2 + c[i, 3]*theta_y**2 + c[i, 4]*theta_x + c[i, 5]*theta_y + c[i, 6]


ISSUE - WANT TO SOLVE FOR THETA_X AND THETA_Y NOT ALL 6

### Solve Matrix Equation